In [334]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import math
from datar.apis.dplyr import if_else
import time
from IPython.display import clear_output
import warnings
import plotly.express as px
import kaleido
%matplotlib qt

In [ ]:
pce_files = os.listdir('/Volumes/FRASER USB/MRes/Data/2022/PCE events')
pce_files = pd.Series(pce_files)

penguins = ['C42', 'C52', 'C56', 'C65', 'C61', 'C67', 'C68', 'C120']

for p in penguins:

    pce = pd.DataFrame(columns=['PCE', 'vid', 'frame', 'vid_time', 'BEHAV', 'Timestamp'])

    penguin_current = pd.read_csv('/Volumes/FRASER USB/MRes/Data/2022/EM results classified/'+str(p)+'_25Hz_EM_classifications.csv')

    penguin_pce = pce_files[(pce_files.str.contains(p) == True) & (pce_files.str.contains('_'+str(p)) == False)]

    print(f'Current penguin: {p}')

    for i in penguin_pce:
        print(f'Loading {i}')
        pce_current = pd.read_csv('/Volumes/FRASER USB/MRes/Data/2022/PCE events/'+i)
        pce_current = pd.DataFrame(pce_current)
        pce = pd.concat([pce, pce_current])

    globals()[f'{p}_PCE'] = pd.merge(penguin_current, pce, on='Timestamp', how='left')

In [7]:
C42_PCE['Timestamp'] = pd.to_datetime(C42_PCE['Timestamp'], dayfirst=True)
C52_PCE['Timestamp'] = pd.to_datetime(C52_PCE['Timestamp'], dayfirst=True)
C56_PCE['Timestamp'] = pd.to_datetime(C56_PCE['Timestamp'], dayfirst=True)
C61_PCE['Timestamp'] = pd.to_datetime(C61_PCE['Timestamp'], dayfirst=True)
C65_PCE['Timestamp'] = pd.to_datetime(C65_PCE['Timestamp'], dayfirst=True)
C67_PCE['Timestamp'] = pd.to_datetime(C67_PCE['Timestamp'], dayfirst=True)
C68_PCE['Timestamp'] = pd.to_datetime(C68_PCE['Timestamp'], dayfirst=True)
C120_PCE['Timestamp'] = pd.to_datetime(C120_PCE['Timestamp'], dayfirst=True)

In [8]:
C42_start = pd.to_datetime('04/01/2018 00:11:23', dayfirst=True)
C42_end = pd.to_datetime('04/01/2018 03:34:16', dayfirst=True)

C52_start = pd.to_datetime('06/01/2018 00:10:19', dayfirst=True)
C52_end = pd.to_datetime('06/01/2018 03:20:02', dayfirst=True)

C56_start = pd.to_datetime('07/01/2018 01:15:21', dayfirst=True)
C56_end = pd.to_datetime('07/01/2018 02:50:21', dayfirst=True)

C61_start = pd.to_datetime('08/01/2018 12:05:20', dayfirst=True)
C61_end = pd.to_datetime('08/01/2018 14:58:01', dayfirst=True)

C65_start = pd.to_datetime('09/01/2018 22:55:22', dayfirst=True)
C65_end = pd.to_datetime('10/01/2018 03:02:04', dayfirst=True)

C67_start = pd.to_datetime('11/01/2018 22:45:24', dayfirst=True)
C67_end = pd.to_datetime('12/01/2018 01:50:16', dayfirst=True)

C68_start = pd.to_datetime('11/01/2018 22:45:24', dayfirst=True)
C68_end = pd.to_datetime('12/01/2018 03:07:50', dayfirst=True)

C120_start = pd.to_datetime('22/12/2018 14:30:11', dayfirst=True)
C120_end = pd.to_datetime('22/12/2018 17:46:25', dayfirst=True)

In [9]:
C42 = pd.DataFrame(C42_PCE[(C42_PCE.Timestamp > C42_start) & (C42_PCE.Timestamp < C42_end)])
C52 = pd.DataFrame(C52_PCE[(C52_PCE.Timestamp > C52_start) & (C52_PCE.Timestamp < C52_end)])
C56 = pd.DataFrame(C56_PCE[(C56_PCE.Timestamp > C56_start) & (C56_PCE.Timestamp < C56_end)])
C61 = pd.DataFrame(C61_PCE[(C61_PCE.Timestamp > C61_start) & (C61_PCE.Timestamp < C61_end)])
C65 = pd.DataFrame(C65_PCE[(C65_PCE.Timestamp > C65_start) & (C65_PCE.Timestamp < C65_end)])
C67 = pd.DataFrame(C67_PCE[(C67_PCE.Timestamp > C67_start) & (C67_PCE.Timestamp < C67_end)])
C68 = pd.DataFrame(C68_PCE[(C68_PCE.Timestamp > C68_start) & (C68_PCE.Timestamp < C68_end)])
C120 = pd.DataFrame(C120_PCE[(C120_PCE.Timestamp > C120_start) & (C120_PCE.Timestamp < C120_end)])

In [10]:
C42['Timestampvideo'] = C42['Timestamp'] + pd.Timedelta(hours = 9, minutes = 59, seconds = 43)
C52['Timestampvideo'] = C52['Timestamp'] + pd.Timedelta(hours = 9, minutes = 59, seconds = 46)
C56['Timestampvideo'] = C56['Timestamp'] + pd.Timedelta(hours = 9, minutes = 59, seconds = 46)
C61['Timestampvideo'] = C61['Timestamp'] + pd.Timedelta(hours = 9, minutes = 59, seconds = 45)
C65['Timestampvideo'] = C65['Timestamp'] + pd.Timedelta(hours = 9, minutes = 59, seconds = 42)
C67['Timestampvideo'] = C67['Timestamp'] + pd.Timedelta(hours = 9, minutes = 59, seconds = 39)
C68['Timestampvideo'] = C68['Timestamp'] + pd.Timedelta(hours = 9, minutes = 59, seconds = 40)
C120['Timestampvideo'] = C120['Timestamp'] + pd.Timedelta(hours = 9, minutes = 59, seconds = 53)

In [11]:
logger = pd.to_datetime('22/12/2018 14:30:11', dayfirst=True)
video = pd.to_datetime('23/12/2018 00:30:04', dayfirst=True)
video - logger

Timedelta('0 days 09:59:53')

In [12]:
# create dive clusters
for p in penguins:

    current_penguin = globals()[p]

    current_penguin['dive_cluster'] = np.nan
    is_dive = current_penguin['depth25Hz'] > 2

    start_new_dive = (is_dive & (is_dive.shift(1) == False)).cumsum()
    current_penguin.loc[is_dive, 'dive_cluster'] = start_new_dive[is_dive]

    globals()[p] = current_penguin


In [13]:
plt.scatter(C65['Timestamp'], C65['depth25Hz'], c=C65['dive_cluster'], s = 1)

In [14]:
'''
Removing dives with low visibility
'''
# C42
C42_rem = ['04/01/2018 11:04:44',
           '04/01/2018 11:07:54',
           '04/01/2018 11:50:43',
           '04/01/2018 11:58:27',
           '04/01/2018 12:01:22',
           '04/01/2018 12:04:10',
           '04/01/2018 12:07:02',
           '04/01/2018 12:09:54',
           '04/01/2018 12:18:31',
           '04/01/2018 12:21:27']
# C52
C52_rem = ['06/01/2018 11:01:09',
           '06/01/2018 11:04:26']
# C56 - none
# C61
C61_rem = ['08/01/2018 22:16:35',
           '08/01/2018 22:19:33',
           '08/01/2018 22:22:27',
           '08/01/2018 22:25:34',
           '08/01/2018 22:28:23',
           '08/01/2018 22:31:42',
           '08/01/2018 22:35:00',
           '08/01/2018 23:39:56',
           '08/01/2018 23:42:51',
           '08/01/2018 23:46:21',
           '08/01/2018 23:50:03',
           '08/01/2018 23:53:40',
           '08/01/2018 23:57:09',
           '09/01/2018 00:00:37',
           '09/01/2018 00:46:15',
           '09/01/2018 00:49:04',
           '09/01/2018 00:52:03',
           '09/01/2018 00:55:14']
# C65
C65_rem = ['10/01/2018 12:25:47']
# C67 - none
# C68 - none
# C120
C120_rem = ['23/12/2018 00:52:47',
            '23/12/2018 00:54:54',
            '23/12/2018 01:01:47',
            '23/12/2018 01:04:13',
            '23/12/2018 01:14:45',
            '23/12/2018 01:17:27',
            '23/12/2018 01:19:21',
            '23/12/2018 01:22:27',
            '23/12/2018 01:25:05',
            '23/12/2018 01:27:54',
            '23/12/2018 01:30:51']

for p in penguins:
    rem = p + '_rem'
    current_penguin = globals()[p]
    current_penguin['low_visibility'] = False
    if rem in globals():
        current_rem = globals()[rem]
        for i in range(len(current_rem)):
            current_time = pd.to_datetime(current_rem[i], dayfirst=True)
            buffered_time = current_time + pd.Timedelta(hours = 0, minutes = 0, seconds = 30)

            diffs = abs(buffered_time - current_penguin['Timestampvideo'])
            closest = diffs.idxmin()
            current_penguin.loc[closest, 'low_visibility'] = True

            low_vis = current_penguin.dive_cluster[current_penguin['low_visibility'] == True].values

            current_penguin.loc[current_penguin['dive_cluster'].isin(low_vis), 'low_visibility'] = True

            globals()[p] = current_penguin



In [15]:
# low visibility dives in red
plt.scatter(C42['Timestamp'], C42['depth25Hz'], s = 1, zorder = 1)
mask = C42['low_visibility'] == True
plt.scatter(C42['Timestamp'][mask], C42['depth25Hz'][mask], c='red', s=5, zorder=2)

In [16]:
# create prediction clusters

for p in penguins:

    current_penguin = globals()[p]

    current_penguin['hunting_cluster'] = np.nan
    is_hunting = current_penguin['States_new'] == 'Hunt'

    start_new_hunt = (is_hunting & (is_hunting.shift(1) == False)).cumsum()
    current_penguin.loc[is_hunting, 'hunting_cluster'] = start_new_hunt[is_hunting]

    globals()[p] = current_penguin


In [17]:
plt.scatter(C42['Timestamp'], C42['depth25Hz'], s = 1, zorder = 2)
mask = C42['low_visibility'] == True
plt.scatter(C42['Timestamp'][mask], C42['depth25Hz'][mask], c='red', s=5, zorder=3)
plt.plot(C42['Timestamp'], C42['Hunting'], c='red', zorder=1)

In [18]:
C42['TP'] = np.where(
    (C42['Hunting'] > 0) & (C42['PCE'].isin(range(1,8))), # range(1,8) is [1,2,3,4,5,6,7]
    True,
    False
)
C42['TN'] = np.where(
    (C42['Hunting'] == 0) & (~C42['PCE'].isin(range(1,8))),
    True,
    False
)
C42['FP'] = np.where(
    (C42['Hunting'] > 0) & (~C42['PCE'].isin(range(1,8))),
    True,
    False
)
C42['FN'] = np.where(
    (C42['Hunting'] == 0) & (C42['PCE'].isin(range(1,8))),
    True,
    False
)

TP_count = C42['TP'].sum()
TN_count = C42['TN'].sum()
FP_count = C42['FP'].sum()
FN_count = C42['FN'].sum()

print(f'True positives: {TP_count};',
      f'True negatives: {TN_count};',
      f'False positives: {FP_count};',
      f'False negatives: {FN_count}')

True positives: 590; True negatives: 250397; False positives: 53439; False negatives: 32


In [19]:
# adding window size column

for p in penguins:

    current_penguin = globals()[p]

    window_size = (
        current_penguin
        .groupby('hunting_cluster')
        .size()
        .reset_index(name='window_size')
    )

    current_penguin = current_penguin.merge(window_size, on='hunting_cluster', how='left')

    globals()[p] = current_penguin


In [52]:
# 1. window size vs pce count

for p in penguins:

    current_penguin = globals()[p]

    current_penguin = current_penguin.drop(current_penguin[current_penguin.low_visibility == True].index)

    test = (
        current_penguin
        .assign(
            PCE_in_range=current_penguin['PCE'].between(1, 7)
        )
        .reset_index()
        .groupby('hunting_cluster')
        .apply(lambda df: pd.Series({
                        'pce_count': df['PCE_in_range'].sum(),
                        'total_count': len(df)
                    }))
    )

    plt.clf()
    plt.scatter(test['pce_count'], test['total_count']/25, alpha=0.3)
    plt.xlabel('True Events')
    plt.ylabel('Prediction Size (s)')
    plt.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/pce count vs window size/' + p + '_pcecount_windowsize.png')


/var/folders/m2/pp9h0nyd3lqfyng9rfwmg9s00000gn/T/ipykernel_22600/289568595.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: pd.Series({
/var/folders/m2/pp9h0nyd3lqfyng9rfwmg9s00000gn/T/ipykernel_22600/289568595.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: pd.Series({
/var/folders/m2/pp9h0nyd3lqfyng9rfwmg9s00000gn/T/ipykernel_22600/289568595.py:16: FutureWarning: DataFra

In [60]:
# 2. barchart of number of pce events inside and outside of windows

df = pd.DataFrame(columns=['ID', 'Inside', 'Outside'])

inside_comb = 0
outside_comb = 0

for p in penguins:

    current_penguin = globals()[p]

    current_penguin = current_penguin.drop(current_penguin[current_penguin.low_visibility == True].index)

    outside = len(current_penguin[(current_penguin['States_new'] != 'Hunt') & (current_penguin['PCE'].isin(range(1,8)))])
    inside = len(current_penguin[(current_penguin['States_new'] == 'Hunt') & (current_penguin['PCE'].isin(range(1,8)))])

    df_current = pd.DataFrame(columns=['ID', 'Inside', 'Outside'])
    df_current.loc[0, 'ID'] = p
    df_current.loc[0, 'Inside'] = inside
    df_current.loc[0, 'Outside'] = outside

    df = pd.concat([df, df_current])

    inside_comb += inside
    outside_comb += outside

    plt.clf()
    plt.bar('In EM window', (inside / (inside + outside)))
    plt.bar('Not in EM window', (outside / (inside + outside)))
    plt.bar('False positive', )
    plt.ylim(0, 1)
    plt.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/in vs out em window/' + p + '_in_vs_out_EMwindow.png')


# all combined

plt.clf()
plt.bar('In EM window', (inside_comb / (inside_comb + outside_comb)))
plt.bar('Not in EM window', (outside_comb / (inside_comb + outside_comb)))
plt.ylim(0, 1)
plt.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/in vs out em window/allpenguins_in_vs_out_EMwindow.png')


In [77]:
# 2.1. faceted plot of all penguins

df['Inside_norm'] = df['Inside'] / (df['Inside'] + df['Outside'])
df['Outside_norm'] = df['Outside'] / (df['Inside'] + df['Outside'])

df_long = df.melt(id_vars='ID', value_vars=['Inside_norm', 'Outside_norm'],
                  var_name='Category', value_name='Value')

name_map = {
    'Inside_norm': 'In EM window',
    'Outside_norm': 'Not in EM window'
}
df_long['Category'] = df_long['Category'].replace(name_map)

g = sns.FacetGrid(df_long, col='ID', col_wrap=3, sharey=True)
g.map_dataframe(sns.barplot, x='Category', y='Value',
                hue='Category',
                palette={'In EM window': 'tab:blue', 'Not in EM window': 'tab:orange'})
g.set_axis_labels("", "")
g.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/in vs out em window/faceted_in_vs_out_EMwindow.png')

In [116]:
# 3. window size vs prey type

pce_index_all = pd.DataFrame(columns=['1',
                                      '2',
                                      '3',
                                      '4',
                                      '5',
                                      '6',
                                      '7',
                                      'total_count',
                                      'ID'])

for p in penguins:

    current_penguin = globals()[p]

    current_penguin = current_penguin.drop(current_penguin[current_penguin.low_visibility == True].index)

    pce_index = (
        current_penguin
        .reset_index()
        .groupby('hunting_cluster')
        .apply(lambda df: pd.Series({
            '1': (df['PCE'] == 1).sum(),
            '2': (df['PCE'] == 2).sum(),
            '3': (df['PCE'] == 3).sum(),
            '4': (df['PCE'] == 4).sum(),
            '5': (df['PCE'] == 5).sum(),
            '6': (df['PCE'] == 6).sum(),
            '7': (df['PCE'] == 7).sum(),
            'total_count': len(df)
        }))
    )

    pce_index['ID'] = p

    pce_index_all = pd.concat([pce_index_all, pce_index])

    plt.clf()
    plt.scatter(pce_index['1'], pce_index['total_count']/25, alpha=0.3, label='Potential PCE')
    plt.scatter(pce_index['2'], pce_index['total_count']/25, alpha=0.3, label = 'Confirmed PCE')
    plt.scatter(pce_index['3'], pce_index['total_count']/25, alpha=0.3, label = 'Krill (individual)')
    plt.scatter(pce_index['4'], pce_index['total_count']/25, alpha=0.3, label = 'Krill (swarm)')
    plt.scatter(pce_index['5'], pce_index['total_count']/25, alpha=0.3, label = 'Fish')
    plt.scatter(pce_index['6'], pce_index['total_count']/25, alpha=0.3, label = 'Jellyfish')
    plt.scatter(pce_index['7'], pce_index['total_count']/25, alpha=0.3, label = 'Other prey')
    plt.legend(title = '')
    plt.xlabel('True Events')
    plt.ylabel('EM Prediction Size (s)')
    plt.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/prey type vs window size/' + p + '_preytype_windowsize.png')

/var/folders/m2/pp9h0nyd3lqfyng9rfwmg9s00000gn/T/ipykernel_22600/444944179.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: pd.Series({
/var/folders/m2/pp9h0nyd3lqfyng9rfwmg9s00000gn/T/ipykernel_22600/444944179.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: pd.Series({
/var/folders/m2/pp9h0nyd3lqfyng9rfwmg9s00000gn/T/ipykernel_22600/444944179.py:23: FutureWarning: DataFra

In [118]:
# 3.1. all penguins combined, faceted to the different species caught

label_map = {
    '1': 'Potential PCE',
    '2': 'Confirmed PCE',
    '3': 'Krill (individual)',
    '4': 'Krill (swarm)',
    '5': 'Fish',
    '6': 'Jellyfish',
    '7': 'Other prey'
}

pce_long = pce_index_all.melt( # transpose (melt) the dataset
    id_vars=['total_count', 'ID'],
    value_vars=[str(i) for i in range(1, 8)],
    var_name='PCE',
    value_name='count'
)

pce_long['PCE_label'] = pce_long['PCE'].map(label_map) # assign respective labels

pce_long['window_size_s'] = pce_long['total_count'] / 25

pce_long['count'] = pd.to_numeric(pce_long['count'], errors='coerce')
pce_long['window_size_s'] = pd.to_numeric(pce_long['window_size_s'], errors='coerce')

g = sns.FacetGrid(pce_long[pce_long['count'] > 0], col='PCE_label', col_wrap=3, sharey=True)
g.map_dataframe(sns.scatterplot, x='count', y='window_size_s', alpha=1, s=3)
g.map_dataframe(sns.regplot, x='count', y='window_size_s',
                scatter=False, ci=90, color='grey')
g.set_axis_labels("True Events", "EM Prediction Size (s)")
g.set_titles("{col_name}")
g.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/prey type vs window size/allpenguins_preytype_faceted.png')

In [129]:
# 3.2. all prey, faceted to different penguin IDs

# 3.2.1. uncoloured (all labels combined)
g = sns.FacetGrid(pce_long[pce_long['count'] > 0], col='ID', col_wrap=3, sharey=True)
g.map_dataframe(sns.scatterplot, x='count', y='window_size_s', alpha=1, s=3)
g.set_axis_labels("True Events", "EM Prediction Size (s)")
g.set_titles("{col_name}")
g.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/pce count vs window size/pcecount_windowsize_faceted.png')

# 3.2.2. coloured (distinct PCE labels)
plt.clf()
g = sns.FacetGrid(pce_long[pce_long['count'] > 0], col='ID', col_wrap=3, hue='PCE_label', sharey=True)
g.map_dataframe(sns.scatterplot, x='count', y='window_size_s', alpha=0.5, s=10)
g.set_axis_labels("True Events", "EM Prediction Size (s)")
g.set_titles("{col_name}")
g.add_legend(title='PCE label')
g.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/prey type vs window size/preytype_penguins_faceted.png')


In [162]:
# 3.3. single prey plots, faceted to different penguin IDs
# do this later

# for i in range(1, 7):
#
#     current_label = pce_long[pce_long['PCE'] == str(i)]['PCE_label'].iloc[0]
#
#     g = sns.FacetGrid(pce_long[(pce_long['count'] > 0) & pce_long['PCE'] == i], col='ID', col_wrap=3, sharey=True)
#     g.map_dataframe(sns.scatterplot, x='count', y='window_size_s', alpha=1, s=3)
#     g.set_axis_labels("True Events", "EM Prediction Size (s)")
#     g.set_titles("{col_name}")
#     g.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/prey type vs window size/single prey plots faceted by penguin id/' + current_label + '.png')

ValueError: Number of rows must be a positive integer, not 0

In [164]:
# 4. time spent hunting per dive vs prey type

dive_index_all = pd.DataFrame(columns=['1',
                                      '2',
                                      '3',
                                      '4',
                                      '5',
                                      '6',
                                      '7',
                                      'total_count',
                                      'ID'])

for p in penguins:

    current_penguin = globals()[p]

    current_penguin = current_penguin.drop(current_penguin[current_penguin.low_visibility == True].index)

    dive_index = (
        current_penguin
        .reset_index()
        .groupby('dive_cluster')
        .apply(lambda df: pd.Series({
            '1': (df['PCE'] == 1).sum(),
            '2': (df['PCE'] == 2).sum(),
            '3': (df['PCE'] == 3).sum(),
            '4': (df['PCE'] == 4).sum(),
            '5': (df['PCE'] == 5).sum(),
            '6': (df['PCE'] == 6).sum(),
            '7': (df['PCE'] == 7).sum(),
            'total_count': len(df)
        }))
    )

    dive_index['ID'] = p

    dive_index_all = pd.concat([dive_index_all, dive_index])


label_map = {
    '1': 'Potential PCE',
    '2': 'Confirmed PCE',
    '3': 'Krill (individual)',
    '4': 'Krill (swarm)',
    '5': 'Fish',
    '6': 'Jellyfish',
    '7': 'Other prey'
}

dive_long = dive_index_all.melt( # transpose (melt) the dataset
    id_vars=['total_count', 'ID'],
    value_vars=[str(i) for i in range(1, 8)],
    var_name='PCE',
    value_name='count'
)

dive_long['PCE_label'] = dive_long['PCE'].map(label_map) # assign respective labels

dive_long['window_size_s'] = dive_long['total_count'] / 25

dive_long['count'] = pd.to_numeric(dive_long['count'], errors='coerce')
dive_long['window_size_s'] = pd.to_numeric(dive_long['window_size_s'], errors='coerce')

g = sns.FacetGrid(dive_long[dive_long['count'] > 0], col='PCE_label', col_wrap=3, sharey=True)
g.map_dataframe(sns.scatterplot, x='count', y='window_size_s', alpha=1, s=3)
g.set_axis_labels("True Events", "Dive Length (s)")
g.set_titles("{col_name}")
g.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/prey type vs dive size/allpenguins_preytype_faceted_dives.png')

dive_cluster
38.0    3300
20.0    3125
39.0    3100
19.0    3075
18.0    2900
35.0    2700
34.0    2650
14.0    2600
9.0     2525
15.0    2450
7.0     2325
4.0     2325
5.0     2300
1.0     2300
2.0     2300
8.0     2250
10.0    2225
3.0     2225
6.0     2075
11.0    2050
37.0    1175
0.0      674
29.0     275
30.0     250
33.0     200
36.0     175
28.0     100
32.0      75
31.0      50
Name: count, dtype: int64

In [222]:
# 5. validated window length (s) vs unvalidated window length (s)

warnings.filterwarnings('ignore')
match_all = pd.DataFrame(columns=['hunting_cluster', 'pce_occurred', 'total_count'])

for p in penguins:

    current_penguin = globals()[p]

    current_penguin = current_penguin.drop(current_penguin[current_penguin.low_visibility == True].index)

    match = (
        current_penguin
         .groupby('hunting_cluster', as_index=False)
         .apply(lambda df: pd.Series({
            'pce_occurred': (df['PCE'].isin(range(1,8))).sum(),
            'total_count': len(df)
        }))
    )

    match_all = pd.concat([match_all, match])

validated = match[match['pce_occurred'] > 0]
unvalidated = match[match['pce_occurred'] == 0]

validated['validation'] = 'Validated'
unvalidated['validation'] = 'Unvalidated'

combined = pd.concat([validated, unvalidated])

fig = px.box(
    combined,
    x='validation',
    y=combined['total_count'] / 25,
    points='all',
    labels={'validation': '', 'y': 'EM Prediction Size (s)'}
)

fig.show()

In [214]:
validated_norm = validated['total_count'].sum() / (validated['total_count'].sum() + unvalidated['total_count'].sum())
unvalidated_norm = unvalidated['total_count'].sum() / (validated['total_count'].sum() + unvalidated['total_count'].sum())

plt.bar('PCE', validated_norm)
plt.bar('Non-PCE', unvalidated_norm)
plt.show()

plt.savefig('/Volumes/FRASER USB/MRes/Data/2022/Validation/basic plots/validated time vs non-validated time/validated_vs_non_validated_proportion.png')

In [211]:
print(f'PCE windows: {len(validated)}')
print(f'Non-PCE windows: {len(unvalidated)}')

PCE windows: 128
Non-PCE windows: 1106


In [ ]:
# ----------------------------------

# more in-depth analyses of the EM

# ----------------------------------

In [322]:
# calculate distance between false negative annotations and closest prediction windows

nearest_all = pd.DataFrame(columns = ['pce_idx', 'pred_idx', 'distance', 'penguin'])

for p in penguins:

    current_penguin = globals()[p]

    predictions = current_penguin.index[current_penguin['States_new'] == 'Hunt']
    out_pce = current_penguin.index[(current_penguin['PCE'].isin(range(1,8))) & (current_penguin['States_new'] != 'Hunt')]

    predictions_df = pd.DataFrame({'pred_idx': predictions})
    out_pce_df = pd.DataFrame({'pce_idx': out_pce})

    nearest_df = pd.merge_asof(out_pce_df, predictions_df, left_on='pce_idx', right_on='pred_idx', direction = 'nearest')

    nearest_df['distance'] = (nearest_df['pce_idx'] - nearest_df['pred_idx'])

    nearest_df['penguin'] = p

    nearest_all = pd.concat([nearest_all, nearest_df])




In [320]:
plt.hist(nearest_all[(nearest_all['distance'] < 50) & (nearest_all['distance'] > -50)]['distance'], bins = np.arange(-50, 51+1) - 0.5, density = True)
plt.xlim(-50, 50)
plt.ylabel('Density')
plt.xlabel('Number of frames between an annotated PCE frame and closest EM prediction')
nearest_all['distance'].value_counts()


distance
 1       153
-1       128
 2        97
-2        89
-3        73
        ... 
-134       1
 1146      1
 1354      1
-131       1
 89        1
Name: count, Length: 73, dtype: int64

In [340]:
nearest_current = nearest_all[nearest_all['penguin'] == 'C65']

plt.hist(nearest_current[(nearest_current['distance'] < 50) & (nearest_current['distance'] > -50)]['distance'], bins = np.arange(-50, 51+1) - 0.5, density = True)
plt.xlim(-50, 50)
plt.ylabel('Density')
plt.xlabel('Number of frames between an annotated PCE frame and closest EM prediction')
nearest_current['distance'].value_counts()


distance
1      93
-1     77
2      53
-2     44
3      42
-3     36
5      24
-5     23
4      22
6      19
-4     17
-6     12
-7     12
7      11
8       8
10      6
12      6
-8      5
9       4
11      4
13      3
18      3
-9      3
15      2
-18     2
-19     1
-16     1
-14     1
-13     1
-21     1
-15     1
-11     1
25      1
-10     1
-12     1
Name: count, dtype: int64

In [339]:
np.quantile(abs(nearest_all['distance']), 0.9)

np.float64(11.0)

In [ ]:
# ----------------------------------

# Statistical testing - correlations

# ----------------------------------

In [338]:
# Saving the penguin files

for p in penguins:

    current_penguin = globals()[p]

    current_penguin.to_csv('/Volumes/FRASER USB/MRes/Data/2022/Python-formatted EM files with PCE/' + p + '.csv', index=False)
